In [1]:
!pip install streamlit langchain langchain-openai google-colab


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.1/10.1 MB 65.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.0/75.0 kB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 90.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 55.4 MB/s eta 0:00:00


**CHATBOT V1**

In [ ]:
%%writefile app.py
import os
from google.colab import userdata
import streamlit as st
from langchain_openai import ChatOpenAI

# Cargar API key
openai_api_key = userdata.get('OPENAI_API_KEY')
# Inicializar LLM
llm = ChatOpenAI(model="gpt-5", temperature=0, api_key=openai_api_key)

st.title("U-Tutor, tu asistente universitario")

# Mensaje de sistema (contexto para la IA)
messages = [("system", """Eres Jake, un tutor universitario empático y profesional. Ayuda al estudiante con sus dudas, habla como humano y nunca reveles que eres una IA.""" )]

# Inicializar historial de chat
if "messages" not in st.session_state:
    st.session_state.messages = []

# Mostrar historial de chat al recargar la app
for message in st.session_state.messages:
    with st.chat_message(message["role"]):
        st.markdown(message["content"])

# Reaccionar a la entrada del usuario
if prompt := st.chat_input("Escribe tu mensaje..."):
    # Mostrar mensaje del usuario
    st.chat_message("user").markdown(prompt)
    # Agregar mensaje del usuario al historial
    st.session_state.messages.append({"role": "user", "content": prompt})
    messages.append(["human", prompt])

    # Obtener respuesta de Jake
    response = llm.invoke(messages).content

    # Mostrar respuesta del asistente
    with st.chat_message("assistant"):
        st.markdown(response)

    # Agregar respuesta del asistente al historial
    st.session_state.messages.append({"role": "assistant", "content": response})

Overwriting app.py


In [ ]:
!npm install localtunnel

In [ ]:
!streamlit run app.py &>/content/logs.txt & npx localtunnel --port 8501

Busca la Ip externa en el archivo logs.txt esa es el password del tunnel

**CHATBOT V2**

En el archivo chat_manager.py tenemos La Lógica del chat y OpenAI API

In [2]:
%%writefile chat_manager.py
from typing import List, Dict, Any
from langchain_openai import ChatOpenAI


class ChatManager:
    def __init__(self, api_key: str, model: str , temperature: float = 0):
        self.llm = ChatOpenAI(
            model=model,
            temperature=temperature,
            api_key=api_key
        )
        self.system_message = """Eres Jake, un tutor universitario empático y profesional.
        Ayuda al estudiante con sus dudas, habla como humano y nunca reveles que eres una IA."""

    def prepare_messages_for_api(self, messages: List[Dict[str, str]]) -> List[tuple]:
        """Prepara los mensajes para la API de OpenAI"""
        api_messages = [("system", self.system_message)]

        for msg in messages:
            role = "human" if msg["role"] == "user" else "assistant"
            api_messages.append((role, msg["content"]))

        return api_messages

    def get_response(self, messages: List[Dict[str, str]]) -> str:
        """Obtiene una respuesta del modelo de IA"""
        try:
            api_messages = self.prepare_messages_for_api(messages)
            response = self.llm.invoke(api_messages)
            return response.content
        except Exception as e:
            raise Exception(f"Error al obtener respuesta del modelo: {str(e)}")

    def generate_conversation_title(self, first_message: str, max_length: int = 50) -> str:
        """Genera un título para la conversación basado en el primer mensaje"""
        if len(first_message) > max_length:
            return first_message[:max_length].strip() + "..."
        return first_message.strip()

    def validate_message(self, message: str) -> bool:
        """Valida que el mensaje no esté vacío"""
        return message and message.strip()

Writing chat_manager.py


En el archivo ui_components.py encontramos todos los Componentes de interfaz

In [3]:
%%writefile ui_components.py
import streamlit as st
from typing import List, Tuple, Optional
from database_manager import DatabaseManager


class UIComponents:
    def __init__(self, db_manager: DatabaseManager, version: str, model: str):
        self.db_manager = db_manager
        self.version = version
        self.model = model

    def render_sidebar(self) -> Optional[int]:
        """Renderiza el sidebar con gestión de conversaciones"""
        st.sidebar.title("🗂️ Historial de Chats")

        selected_conversation_id = None

        # Botón para nueva conversación
        if st.sidebar.button("➕ Nueva Conversación", use_container_width=True):
            st.session_state.current_conversation_id = None
            st.session_state.messages = []
            st.session_state.editing_title = None
            st.rerun()

        # Mostrar conversaciones existentes
        conversations = self.db_manager.get_conversations()

        if conversations:
            st.sidebar.subheader("Conversaciones guardadas:")

            for conv_id, title, created_at, updated_at in conversations:
                self._render_conversation_item(conv_id, title, created_at)

        # Estadísticas
        self._render_stats()

        return selected_conversation_id

    def _render_conversation_item(self, conv_id: int, title: str, created_at: str):
        """Renderiza un elemento de conversación en el sidebar"""
        # Contenedor para la conversación
        container = st.sidebar.container()

        with container:
            # Verificar si estamos editando el título de esta conversación
            is_editing = (
                hasattr(st.session_state, 'editing_title') and
                st.session_state.editing_title == conv_id
            )

            if is_editing:
                # Modo edición del título
                col1, col2, col3 = st.columns([3, 1, 1])

                with col1:
                    new_title = st.text_input(
                        "Nuevo título:",
                        value=title,
                        key=f"edit_title_{conv_id}",
                        label_visibility="collapsed"
                    )

                with col2:
                    if st.button("✅", key=f"save_{conv_id}", help="Guardar"):
                        if new_title.strip():
                            if self.db_manager.update_conversation_title(conv_id, new_title.strip()):
                                st.success("Título actualizado!")
                                st.session_state.editing_title = None
                                st.rerun()
                            else:
                                st.error("Error al actualizar")
                        else:
                            st.warning("El título no puede estar vacío")

                with col3:
                    if st.button("❌", key=f"cancel_{conv_id}", help="Cancelar"):
                        st.session_state.editing_title = None
                        st.rerun()

            else:
                # Modo normal
                col1, col2, col3 = st.columns([3, 1, 1])

                with col1:
                    if st.button(
                        f"💬 {title}",
                        key=f"conv_{conv_id}",
                        use_container_width=True,
                        help=f"Creado: {created_at[:16]}"
                    ):
                        self._load_conversation(conv_id)

                with col2:
                    if st.button("✏️", key=f"edit_{conv_id}", help="Editar título"):
                        st.session_state.editing_title = conv_id
                        st.rerun()

                with col3:
                    if st.button("🗑️", key=f"del_{conv_id}", help="Eliminar conversación"):
                        self._delete_conversation_with_confirmation(conv_id)

    def _load_conversation(self, conv_id: int):
        """Carga una conversación específica"""
        st.session_state.current_conversation_id = conv_id
        st.session_state.editing_title = None

        # Cargar mensajes de la conversación
        messages_data = self.db_manager.load_conversation_messages(conv_id)
        st.session_state.messages = []

        for role, content, _ in messages_data:
            st.session_state.messages.append({"role": role, "content": content})

        st.rerun()

    def _delete_conversation_with_confirmation(self, conv_id: int):
        """Elimina una conversación con confirmación"""
        # Crear clave única para el estado de confirmación
        confirm_key = f"confirm_delete_{conv_id}"

        if confirm_key not in st.session_state:
            st.session_state[confirm_key] = False

        if not st.session_state[confirm_key]:
            st.session_state[confirm_key] = True
            st.rerun()
        else:
            # Eliminar la conversación
            if self.db_manager.delete_conversation(conv_id):
                # Si la conversación eliminada era la activa, resetear
                if (hasattr(st.session_state, 'current_conversation_id') and
                    st.session_state.current_conversation_id == conv_id):
                    st.session_state.current_conversation_id = None
                    st.session_state.messages = []

                # Limpiar estado de confirmación
                del st.session_state[confirm_key]
                st.success("Conversación eliminada")
                st.rerun()
            else:
                st.error("Error al eliminar la conversación")

    def _render_stats(self):
        """Renderiza estadísticas en el sidebar"""
        st.sidebar.markdown("---")
        st.sidebar.markdown("### ℹ️ Información")

        stats = self.db_manager.get_conversation_stats()

        st.sidebar.markdown(f"""
        - **Modelo**: {self.model.upper()}
        - **Versión**: U-Tutor v{self.version}
        - **Funciones**:
          - ✅ Historial persistente
          - ✅ Múltiples conversaciones
          - ✅ Editar títulos
          - ✅ Eliminar conversaciones
          - ✅ Continuar chats anteriores
        """)

        if stats['total_conversations'] > 0:
            st.sidebar.markdown(f"**Total de conversaciones**: {stats['total_conversations']}")
            st.sidebar.markdown(f"**Total de mensajes**: {stats['total_messages']}")

    def render_main_chat_area(self):
        """Renderiza el área principal de chat"""
        st.title(f"🎓 U-Tutor v{self.version} - Tu asistente universitario")

        # Mostrar información de la conversación actual
        if hasattr(st.session_state, 'current_conversation_id') and st.session_state.current_conversation_id:
            conversation = self.db_manager.get_conversation_by_id(st.session_state.current_conversation_id)
            if conversation:
                st.info(f"📝 Conversación: **{conversation[1]}** (#{conversation[0]})")
        else:
            st.info("💭 Nueva conversación - Escribe tu primer mensaje para comenzar")

    def render_chat_messages(self, messages: List[dict]):
        """Renderiza los mensajes del chat"""
        for message in messages:
            with st.chat_message(message["role"]):
                st.markdown(message["content"])

    def show_error(self, message: str):
        """Muestra un mensaje de error"""
        st.error(message)

    def show_success(self, message: str):
        """Muestra un mensaje de éxito"""
        st.success(message)

    def show_spinner(self, text: str = "Procesando..."):
        """Muestra un spinner con texto"""
        return st.spinner(text)

Writing ui_components.py


En el archivo database_manager.py tenemos toda la Gestión de SQLite

In [4]:
%%writefile database_manager.py

import sqlite3
from datetime import datetime
from typing import List, Tuple, Optional

class DatabaseManager:
    def __init__(self, db_path: str = "chat_history.db"):
        self.db_path = db_path
        self.init_database()

    def init_database(self):
        """Inicializa la base de datos SQLite"""
        conn = sqlite3.connect(self.db_path)
        cursor = conn.cursor()

        # Crear tabla para conversaciones
        cursor.execute('''
            CREATE TABLE IF NOT EXISTS conversations (
                id INTEGER PRIMARY KEY AUTOINCREMENT,
                title TEXT NOT NULL,
                created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP,
                updated_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP
            )
        ''')

        # Crear tabla para mensajes
        cursor.execute('''
            CREATE TABLE IF NOT EXISTS messages (
                id INTEGER PRIMARY KEY AUTOINCREMENT,
                conversation_id INTEGER,
                role TEXT NOT NULL,
                content TEXT NOT NULL,
                timestamp TIMESTAMP DEFAULT CURRENT_TIMESTAMP,
                FOREIGN KEY (conversation_id) REFERENCES conversations (id)
            )
        ''')

        conn.commit()
        conn.close()

    def create_conversation(self, title: str) -> int:
        """Crea una nueva conversación"""
        conn = sqlite3.connect(self.db_path)
        cursor = conn.cursor()

        cursor.execute(
            "INSERT INTO conversations (title) VALUES (?)",
            (title,)
        )

        conversation_id = cursor.lastrowid
        conn.commit()
        conn.close()

        return conversation_id

    def get_conversations(self) -> List[Tuple]:
        """Obtiene todas las conversaciones ordenadas por fecha de actualización"""
        conn = sqlite3.connect(self.db_path)
        cursor = conn.cursor()

        cursor.execute('''
            SELECT id, title, created_at, updated_at
            FROM conversations
            ORDER BY updated_at DESC
        ''')

        conversations = cursor.fetchall()
        conn.close()

        return conversations

    def get_conversation_by_id(self, conversation_id: int) -> Optional[Tuple]:
        """Obtiene una conversación específica por ID"""
        conn = sqlite3.connect(self.db_path)
        cursor = conn.cursor()

        cursor.execute(
            "SELECT id, title, created_at, updated_at FROM conversations WHERE id = ?",
            (conversation_id,)
        )

        conversation = cursor.fetchone()
        conn.close()

        return conversation

    def update_conversation_title(self, conversation_id: int, new_title: str) -> bool:
        """Actualiza el título de una conversación"""
        conn = sqlite3.connect(self.db_path)
        cursor = conn.cursor()

        cursor.execute(
            "UPDATE conversations SET title = ?, updated_at = CURRENT_TIMESTAMP WHERE id = ?",
            (new_title, conversation_id)
        )

        success = cursor.rowcount > 0
        conn.commit()
        conn.close()

        return success

    def save_message(self, conversation_id: int, role: str, content: str):
        """Guarda un mensaje en la base de datos"""
        conn = sqlite3.connect(self.db_path)
        cursor = conn.cursor()

        # Insertar mensaje
        cursor.execute(
            "INSERT INTO messages (conversation_id, role, content) VALUES (?, ?, ?)",
            (conversation_id, role, content)
        )

        # Actualizar timestamp de la conversación
        cursor.execute(
            "UPDATE conversations SET updated_at = CURRENT_TIMESTAMP WHERE id = ?",
            (conversation_id,)
        )

        conn.commit()
        conn.close()

    def load_conversation_messages(self, conversation_id: int) -> List[Tuple]:
        """Carga el historial de mensajes de una conversación"""
        conn = sqlite3.connect(self.db_path)
        cursor = conn.cursor()

        cursor.execute('''
            SELECT role, content, timestamp
            FROM messages
            WHERE conversation_id = ?
            ORDER BY timestamp ASC
        ''', (conversation_id,))

        messages = cursor.fetchall()
        conn.close()

        return messages

    def delete_conversation(self, conversation_id: int) -> bool:
        """Elimina una conversación y todos sus mensajes"""
        conn = sqlite3.connect(self.db_path)
        cursor = conn.cursor()

        # Eliminar mensajes primero (por la foreign key)
        cursor.execute("DELETE FROM messages WHERE conversation_id = ?", (conversation_id,))

        # Luego eliminar la conversación
        cursor.execute("DELETE FROM conversations WHERE id = ?", (conversation_id,))

        success = cursor.rowcount > 0
        conn.commit()
        conn.close()

        return success

    def get_conversation_stats(self) -> dict:
        """Obtiene estadísticas de las conversaciones"""
        conn = sqlite3.connect(self.db_path)
        cursor = conn.cursor()

        # Total de conversaciones
        cursor.execute("SELECT COUNT(*) FROM conversations")
        total_conversations = cursor.fetchone()[0]

        # Total de mensajes
        cursor.execute("SELECT COUNT(*) FROM messages")
        total_messages = cursor.fetchone()[0]

        # Conversación más reciente
        cursor.execute("""
            SELECT title, updated_at
            FROM conversations
            ORDER BY updated_at DESC
            LIMIT 1
        """)
        latest_conversation = cursor.fetchone()

        conn.close()

        return {
            'total_conversations': total_conversations,
            'total_messages': total_messages,
            'latest_conversation': latest_conversation
        }

Writing database_manager.py


El archivo main.py es nuestra Aplicación principal desde donde ejecutamos el codigo

In [5]:
%%writefile main.py

import os
from dotenv import load_dotenv
import streamlit as st

# Importar nuestros módulos
from database_manager import DatabaseManager
from chat_manager import ChatManager
from ui_components import UIComponents

# Cargar variables de entorno
load_dotenv()

class UTutorApp:
    def __init__(self):

        self.version = os.getenv("VERSION", "1.0")
        self.api_key = os.getenv("OPENAI_API_KEY")
        self.model = os.getenv("MODEL", "gpt-4")


        # Configuración de la aplicación
        st.set_page_config(
            page_title=f"U-Tutor v{self.version}",
            page_icon="🎓",
            layout="wide"
        )

        # Inicializar componentes
        self.db_manager = DatabaseManager()
        self.ui_components = UIComponents(self.db_manager, self.version, self.model)

        if not self.api_key:
            st.error("❌ Por favor, configura tu OPENAI_API_KEY en el archivo .env")
            st.stop()

        self.chat_manager = ChatManager(self.api_key, self.model)

        # Inicializar estado de la sesión
        self._init_session_state()

    def _init_session_state(self):
        """Inicializa el estado de la sesión"""
        if "messages" not in st.session_state:
            st.session_state.messages = []

        if "current_conversation_id" not in st.session_state:
            st.session_state.current_conversation_id = None

        if "editing_title" not in st.session_state:
            st.session_state.editing_title = None

    def run(self):
        """Ejecuta la aplicación principal"""
        # Renderizar sidebar
        self.ui_components.render_sidebar()

        # Renderizar área principal de chat
        self.ui_components.render_main_chat_area()

        # Mostrar historial de mensajes
        self.ui_components.render_chat_messages(st.session_state.messages)

        # Manejar input del usuario
        self._handle_user_input()

    def _handle_user_input(self):
        """Maneja la entrada del usuario y genera respuestas"""
        if prompt := st.chat_input("Escribe tu mensaje..."):
            # Validar mensaje
            if not self.chat_manager.validate_message(prompt):
                self.ui_components.show_error("Por favor, escribe un mensaje válido.")
                return

            # Si es una nueva conversación, crearla
            if st.session_state.current_conversation_id is None:
                conversation_title = self.chat_manager.generate_conversation_title(prompt)
                st.session_state.current_conversation_id = self.db_manager.create_conversation(conversation_title)

            # Mostrar mensaje del usuario
            st.chat_message("user").markdown(prompt)

            # Agregar mensaje del usuario al historial de la sesión
            st.session_state.messages.append({"role": "user", "content": prompt})

            # Guardar mensaje del usuario en la base de datos
            self.db_manager.save_message(
                st.session_state.current_conversation_id,
                "user",
                prompt
            )

            # Generar y mostrar respuesta del asistente
            self._generate_assistant_response()

    def _generate_assistant_response(self):
        """Genera y muestra la respuesta del asistente"""
        with st.chat_message("assistant"):
            with self.ui_components.show_spinner("Jake está pensando..."):
                try:
                    # Obtener respuesta del modelo
                    response = self.chat_manager.get_response(st.session_state.messages)

                    # Mostrar respuesta
                    st.markdown(response)

                    # Agregar respuesta al historial de la sesión
                    st.session_state.messages.append({"role": "assistant", "content": response})

                    # Guardar respuesta en la base de datos
                    self.db_manager.save_message(
                        st.session_state.current_conversation_id,
                        "assistant",
                        response
                    )

                except Exception as e:
                    error_message = f"Error al obtener respuesta: {str(e)}"
                    self.ui_components.show_error(error_message)
                    st.error(f"Detalles técnicos: {e}")


def main():
    """Función principal"""
    app = UTutorApp()
    app.run()


if __name__ == "__main__":
    main()

Writing main.py
